<a href="https://colab.research.google.com/github/minhloc29/minhloc/blob/main/Face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip data.zip


Archive:  data.zip
replace data/anchor/.ipynb_checkpoints/14bdc340-4a30-11ef-b5cf-4cebbd9472ba-checkpoint.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt
import random
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
#1. MOVE THE UNLABELLED IMAGE TO THE NEGATIVE FOLDER
# for image in os.listdir('unlabelled_image'):
#     image_path = os.path.join('unlabelled_image', image)
#     new_image_path = os.path.join('data', 'negative', image)
#     os.replace(image_path, new_image_path)
# folder: 3445 images
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')


In [ ]:
!unzip data.zip

In [ ]:
#DATA AUGMENTATION
def data_aug(img):
    data = []
    for i in range(9):
        img = tf.image.stateless_random_brightness(img, max_delta=0.02, seed=(1,2))
        img = tf.image.stateless_random_contrast(img, lower=0.6, upper=1, seed=(1,3))
        # img = tf.image.stateless_random_crop(img, size=(20,20,3), seed=(1,2))
        img = tf.image.stateless_random_flip_left_right(img, seed=(np.random.randint(100),np.random.randint(100)))
        img = tf.image.stateless_random_jpeg_quality(img, min_jpeg_quality=90, max_jpeg_quality=100, seed=(np.random.randint(100),np.random.randint(100)))
        img = tf.image.stateless_random_saturation(img, lower=0.9,upper=1, seed=(np.random.randint(100),np.random.randint(100)))
        data.append(img)
    return data
test_img = cv2.imread(os.path.join(POS_PATH, '8da1f394-4a30-11ef-8534-4cebbd9472ba.jpg'))
data = data_aug(test_img)
len(data)

In [ ]:
#create dataset
anchor = tf.data.Dataset.list_files(ANC_PATH + '/*.jpg').take(300)
positive = tf.data.Dataset.list_files(POS_PATH + '/*.jpg').take(300)
negative = tf.data.Dataset.list_files(NEG_PATH + '/*.jpg').take(300)
img = os.path.join(POS_PATH, '8d4907df-4a30-11ef-bd8c-4cebbd9472ba.jpg')
#RECSALE AND RESIZE IMAGE
def preprocess(img_path):
    img = tf.io.read_file(img_path)
    img_num = tf.image.decode_jpeg(img, channels=3)
    img_num = tf.image.resize(img_num, (100, 100))
    img_num = img_num / 255.0
    return img_num
anchor = anchor.map(preprocess)
positive = positive.map(preprocess)
negative = negative.map(preprocess)
positives = tf.data.Dataset.zip((anchor, positive,  tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative,  tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)
data = data.shuffle(buffer_size = 1000)
data.cache()
#Create train, val, test
train_data = data.take(round(len(data) *0.7))
test_data = data.skip(round(len(data) *0.7))
train_data = train_data.batch(16)
# len(train_data) count through batches: 27 batches
train_data = train_data.prefetch(8)
test_data = test_data.take(round(len(data) *0.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

In [ ]:
x = train_data.as_numpy_iterator().next()
x[2]

In [ ]:
#MODEL BUILDING
def embedding():
    inp = Input(shape = (100, 100, 3))
    #First
    cv1 = Conv2D(64, (10, 10), activation = 'relu')(inp)
    m1 = MaxPooling2D(64, (2, 2), padding = 'same')(cv1)
    #Second
    c2 = Conv2D(128, (7,7), activation='relu')(m1)
    m2 = MaxPooling2D(64, (2,2), padding='same')(c2)

    # Third block
    c3 = Conv2D(128, (4,4), activation='relu')(m2)
    m3 = MaxPooling2D(64, (2,2), padding='same')(c3)
    # Final embedding block
    c4 = Conv2D(256, (4,4), activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid')(f1)
    result = Model(inputs = inp, outputs = d1, name = 'embedding')
    return result

embed = embedding()
class Dist(Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def call(self, input_embedding, val_embedding):
        return tf.math.abs(input_embedding - val_embedding)

#make a full model, including 2 inputs of train and val, a dist layer
def full_model():
    train_input = Input(shape = (100, 100, 3))
    val_input = Input(shape = (100, 100, 3))
    dist = Dist()
    dist_compute = dist(embed(train_input), embed(val_input))
    result = Dense(1, activation = 'sigmoid')(dist_compute)
    return Model(inputs = [train_input, val_input], outputs = result)
model = full_model()
model.summary()


In [ ]:
X, y, label = test_data.as_numpy_iterator().next()
X

In [ ]:
#LOSS
closs = tf.keras.losses.BinaryCrossentropy()
opt = tf.keras.optimizers.Adam(0.001)
def train_step(batch):
    with tf.GradientTape() as tape:
        #anchor and pos/neg images
        X = batch[:2]
        #labels
        y_true = batch[2]
        #forward pass
        y_pred = model(X, training = True)
        loss = closs(y_true, y_pred)
        grad = tape.gradient(loss, model.trainable_variables)
        opt.apply_gradients(zip(grad, model.trainable_variables))
    return loss

from tensorflow.keras.metrics import Precision, Recall

def train(data, EPOCHS):
    # Loop through epochs
    for epoch in range(1, EPOCHS+1):
        print('\n Epoch {}/{}'.format(epoch, EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data))

        # Creating a metric object
        r = Recall()
        p = Precision()

        # Loop through each batch
        for idx, batch in enumerate(data):
            # Run train step here
            loss = train_step(batch)
            yhat = model.predict(batch[:2])
            r.update_state(batch[2], yhat)
            p.update_state(batch[2], yhat)
            progbar.update(idx+1)
        print(loss.numpy(), r.result().numpy(), p.result().numpy())

        # Save checkpoints
        # if epoch % 10 == 0:
        #     checkpoint.save(file_prefix=checkpoint_prefix)

train(train_data, 20)

In [ ]:
test_inp, test_val, lab = test_data.as_numpy_iterator().next()
y_pred = model.predict([test_inp, test_val])
label_pred = [1 if x > 0.5 else 0 for x in y_pred]
print(lab)
print(label_pred)